In [126]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import chardet
import urllib
from lxml import etree
from lxml import html
from lxml.cssselect import CSSSelector

try:
    from urllib2 import urlopen
except ImportError:
    from urllib.request import urlopen

In [184]:
df0 = pd.read_csv('/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Code/2_Data_Preprocessing/DATA/CSVs/Master_File.csv')

In [209]:
def get_uk_gov_name(row):
    # getting the id from the array
    id = row["id"]
    # getting the domain from the array
    domain = row["domain"]
    # getting the url from the array
    url = row["url"]
    # specifying the path where the html to process is located
    path = '/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Code/1_Data_Collection/DATA/HTMLs/UK/General/{}/{}.html'.format(domain, id)


    # Getting tree for xpath
    htmlparser = etree.HTMLParser()    
    try:
        tree = etree.parse(open(path), htmlparser)
    except UnicodeDecodeError:
        encoding = chardet.detect(rawdata)['encoding']
        tree = etree.parse(open(path, encoding=encoding), htmlparser)

    # organisation 1
    organisation1 = tree.xpath("/html/body/div[@id='wrapper']//span[@class='app-c-publisher-metadata__definition-sentence']/a")
    if type(organisation1) is list and len(organisation1) > 0: 
        row['organisation1'] = organisation1[0].text
    else:
        row['organisation1'] = np.nan

    # organisation 2  
    soup = BeautifulSoup(open(path), "html.parser")
    
    organisation2_element = soup.select_one('.gem-c-organisation-logo__crest')
    
    row['organisation2'] = np.nan
    if organisation2_element is not None:
        row['organisation2'] = organisation2_element.get_text(' ').strip()

    # organisation 3
    organisation3 = tree.xpath("/html/body/div[@id='wrapper']/main//ol[@class='organisation-logos']//img")
    if type(organisation3) is list and len(organisation3) > 0: 
        organisation3 = organisation3[0]
        organisation3 = organisation3.xpath('@alt')
        row['organisation3'] = organisation3[0]
    else:
        row['organisation3'] = np.nan

    # organisation 4
    organisation4 = tree.xpath("/html/body/div[@id='wrapper']//dd[1]/a")
    if type(organisation4) is list and len(organisation4) > 0: 
        row['organisation4'] = organisation4[0].text
    else:
        row['organisation4'] = np.nan


    # organisation 5    
    organisation5_element = soup.select_one('.organisation-logo')

    row['organisation5'] = np.nan

    if organisation5_element is not None:
        row['organisation5'] = organisation5_element.get_text().strip('\n')

    # organisation 6
    organisation6_element = soup.select_one('.organisation > a')
    
    row['organisation6'] = np.nan

    if organisation6_element is not None:
        row['organisation6'] = organisation6_element.get_text().strip()

    # organisation 7
    organisation7_element = soup.select_one('a.organisation-link')
    
    row['organisation7'] = np.nan

    if organisation7_element is not None:
        row['organisation7'] = organisation7_element.get_text().strip('\n')

    return row

In [214]:
df = df0[df0.Organisation=='General Domain']
df = df.apply(get_uk_gov_name, axis=1)
df['final_org'] = np.nan
df.final_org.fillna(df.organisation1, inplace=True)
df.final_org.fillna(df.organisation2, inplace=True)
df.final_org.fillna(df.organisation3, inplace=True)
df.final_org.fillna(df.organisation4, inplace=True)
df.final_org.fillna(df.organisation5, inplace=True)
df.final_org.fillna(df.organisation6, inplace=True)
df.final_org.fillna(df.organisation7, inplace=True)
df.final_org = df.final_org.str.replace('Agile delivery community', 'Agile Delivery Community')
df.final_org = df.final_org.str.replace('Crown Commercial  Service', 'Crown Commercial Service')
df.final_org = df.final_org.str.replace('Defence and Security  Accelerator', 'Ministry of Defence')
df.final_org = df.final_org.str.replace('Department for Business Innovation &Skills', 'Land Registry')
df.final_org = df.final_org.str.replace('Department for Business, Energy & Industrial Strategy', 'Department for Business, Energy and Industrial Strategy')
df.final_org = df.final_org.str.replace('Department for Digital, Culture, Media & Sport', 'Department for Digital, Culture, Media and Sport')
df.final_org = df.final_org.str.replace('Department for Work &  Pensions', 'Department for Work and Pensions')
df.final_org = df.final_org.str.replace('Department of Health & Social Care', 'Department of Health and Social Care')
df.final_org = df.final_org.str.replace('Digital, Data and Technology Profession', 'Civil Service')
df.final_org = df.final_org.str.replace('Driver & Vehicle Standards Agency', 'Driver and Vehicle Standards Agency')
df.final_org = df.final_org.str.replace('Foreign & Commonwealth  Office', 'Foreign and Commonwealth Office')

In [215]:
# adding names that were not found
df.loc[df.url == "https://www.gov.uk/service-manual/communities/agile-delivery-community", "final_org"] = "Agile delivery community"
df.loc[df.url == "https://www.gov.uk/service-manual/communities/product-and-service-community", "final_org"] = "Product and service community"
df.loc[df.url == "https://webarchive.nationalarchives.gov.uk/20130503120649/https://www.gov.uk/service-manual/digital-by-default", "final_org"] = "Government Digital Service"
df.loc[df.url == "https://webarchive.nationalarchives.gov.uk/20190205000341/https://www.gov.uk/government/organisations/land-registry/about/recruitment", "final_org"] = "HM Land Registry"
df.loc[df.url == "https://www.gov.uk/service-manual/service-standard", "final_org"] = "Government Digital Service"
df.loc[df.url == "https://www.gov.uk/service-manual/service-standard/point-8-iterate-and-improve-frequently", "final_org"] = "Government Digital Service"
df.loc[df.url == "https://www.gov.uk/service-manual/service-standard/point-7-use-agile-ways-of-working", "final_org"] = "Government Digital Service"
df.loc[df.url == "https://webarchive.nationalarchives.gov.uk/20190203080135/https://www.gov.uk/service-toolkit", "final_org"] = "Government Digital Service"

# deleting duplicates because of older version on national archives
df = df[~df.url.str.contains('https://webarchive.nationalarchives.gov.uk/20131210192219/https://www.gov.uk/service-manual')] 
df = df[~df.url.str.contains('https://webarchive.nationalarchives.gov.uk/20131213022943/https://www.gov.uk/service-manual/agile')]
df = df[~df.url.str.contains('https://webarchive.nationalarchives.gov.uk/20130503120649/https://www.gov.uk/service-manual/digital-by-default')]
df = df[~df.url.str.contains('https://webarchive.nationalarchives.gov.uk/20170130201946/https://www.gov.uk/service-manual/service-standard')]
df = df[~df.url.str.contains('https://webarchive.nationalarchives.gov.uk/20170130201948/https://www.gov.uk/service-manual')]
df = df[~df.url.str.contains('https://webarchive.nationalarchives.gov.uk/20190201162908/https://www.gov.uk/service-manual/agile-delivery')]
df = df[~df.url.str.contains('https://webarchive.nationalarchives.gov.uk/20190201162908/https://www.gov.uk/service-manual/agile-delivery')]
df = df[~df.url.str.contains('https://webarchive.nationalarchives.gov.uk/20190203055300/https://www.gov.uk/service-manual/communities')]
df = df[~df.url.str.contains('https://webarchive.nationalarchives.gov.uk/20190203055300/https://www.gov.uk/service-manual/communities')]
df = df[~df.url.str.contains('https://webarchive.nationalarchives.gov.uk/20190204220503/https://www.gov.uk/government/organisations/latest?organisations%5B%5D=infrastructure-and-projects-authority')]
df = df[~df.id.str.contains('e580619d-7cb7-31b9-81b5-2fc6a5a79714')]

In [216]:

df.to_csv('/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GitHubRepo/agile-in-government/Code/2_Data_Preprocessing/DATA/CSVs/Org_file.csv')